In [1]:
import numpy as np
import matplotlib.pyplot as plt

import cv2
import os
import itertools

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.models as models
from torchvision import transforms
import torchsummary
import copy

import random
import time

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [2]:

from google.colab import drive
drive.mount('/gdrive', force_remount=True)


Mounted at /gdrive


In [3]:
# Colab specific data loading

# In Google Drive, create a shortcut link of the shared Project folder so that it appears in your root Google Drive directory
# i.e., Right Click Project folder: "Add Shortcut to Drive"

%cd '/gdrive/MyDrive/Project/'


/gdrive/.shortcut-targets-by-id/1ONXXjIHVMg4MHpAdUIdbB4Fr9W33JPYU/Project


In [4]:
# # Data should be in a dir named 'data/'
data = np.load('data/RESISC45_images.npy')
labels = np.load('data/RESISC45_classes.npy')
classes = np.load('data/RESISC45_class_names.npy')


In [5]:

print('Training data shape: ', data.shape)
print('Testing data shape: ', labels.shape)
print('Num Classes', classes.shape)


Training data shape:  (31500, 256, 256, 3)
Testing data shape:  (31500,)
Num Classes (45,)


In [6]:

class addGaussianNoise(object):
    # Transform to add gaussian noise since PyTorch did not have one (that I know of).
    def __init__(self, mean=0.0, std=1.0, p=0.5):
        self.mean = mean
        self.std = std
        self.p = p
      
    def __call__(self, img):
        if torch.rand(1).item() < self.p:
            return img + torch.randn(img.size()) * self.std + self.mean
        return img
        
    def __repr__(self):
        return self.__class__.__name__ + '(mean={0}, std={1}, p={2})'.format(self.mean, self.std, self.p)



In [7]:

# set augmentation parameters for input task
def makeAugParams(task):

    # Reduced list - dropped perspective and blur
    pHF = 0.99 if task[0] == 1 else 0.0
    pVF = 0.99 if task[1] == 1 else 0.0
    rotAng = 359.0 if task[2] == 1 else 0.0
    cropScale = 0.5 if task[3] == 1 else 1.0
    pNoise = 0.99 if task[4] == 1 else 0.0
    satVal = 4.0 if task[5] == 1 else 0.0
    brightVal = 0.5 if task[6] == 1 else 0.0
    contrastVal = 0.9 if task[6] == 1 else 0.0 
    
    # original list
    # pHF = 0.99 if task[0] == 1 else 0.0
    # pVF = 0.99 if task[1] == 1 else 0.0
    # rotAng = 359.0 if task[2] == 1 else 0.0
    # pPersp = 0.99 if task[3] == 1 else 0.0    
    # cropScale = 0.5 if task[4] == 1 else 1.0
    # pNoise = 0.99 if task[5] == 1 else 0.0
    # satVal = 4.0 if task[6] == 1 else 0.0
    # brightVal = 0.5 if task[7] == 1 else 0.0
    # contrastVal = 0.9 if task[7] == 1 else 0.0 
    # blurSigma = (0.01, 2.0) if task[8] == 1 else 1e-9     

    return (pHF, pVF, rotAng, 
            # pPersp, 
            cropScale, pNoise, 
            satVal, brightVal, contrastVal
            # blurSigma
            )
    

In [8]:

def applyAugs(img_batch, current_task):

    # pHF, pVF, rotAng, pPersp, cropScale, pNoise, satVal, brightVal, contrastVal, blurSigma = makeAugParams(current_task)
    pHF, pVF, rotAng, cropScale, pNoise, satVal, brightVal, contrastVal = makeAugParams(current_task)

    transform_list = [transforms.RandomHorizontalFlip(p=pHF),
                      transforms.RandomVerticalFlip(p=pVF),
                      transforms.RandomRotation(rotAng, fill=0.5),
                      # transforms.RandomPerspective(distortion_scale=0.1, p=pPersp, fill=0.5),
                      transforms.RandomResizedCrop(256,
                                                   scale=(cropScale, 1.0),
                                                   ratio=(1.0, 1.0),
                                                   interpolation=transforms.InterpolationMode.BILINEAR),
                      addGaussianNoise(std=0.1, p=pNoise),
                      transforms.ColorJitter(saturation=satVal, hue=0.01),
                      transforms.ColorJitter(brightness=brightVal, contrast=contrastVal)
                      # transforms.GaussianBlur(9, sigma=blurSigma)
                      ]
    
    for i in range(numAugs):
      if current_task[i]==1:
        # t0 = time.time()
        transform = transform_list[i]
        img_batch = transform(img_batch)
        # print(i, time.time() - t0)

    return img_batch



In [9]:

# Create full-factorial combination of the augmentations. Each
# one will be a "task." Each augmentation will only have 2 levels (on and off).
# Hence, there will be 512 tasks if there are 9 augs with 2 levels each.

# First, set number of augmentations per task.
numAugs = 7
augTasks = list(itertools.product([0, 1], repeat=numAugs))


In [10]:

# select random task (for testing)
# in actual implementation we would iterate through the task list
current_task = augTasks[random.randint(0, 128)]


In [11]:

# prepare a minibatch of images
batch_size = 256

d = data[0:batch_size]
d = d / 255.0
d = np.moveaxis(d, 3, 1)
d = torch.as_tensor(d)


In [12]:

# transform minibatch of images
t0 = time.time()
transformed_batch = applyAugs(d, current_task)
print("Time to transform batch of size {:d}: {:0f}".format(len(d), time.time()-t0))


Time to transform batch of size 256: 4.470423


In [ ]:

# plot sample of results
from mpl_toolkits.axes_grid1 import ImageGrid

transformed_batch = transformed_batch.numpy()
transformed_batch = np.moveaxis(transformed_batch, 1, 3)

fig = plt.figure(figsize=(15., 15.))
grid = ImageGrid(fig, 111,  
                 nrows_ncols=(4, 4),  
                 axes_pad=0.1)

for ax, im in zip(grid, transformed_batch[0:16]):
    # Iterating over the grid returns the Axes.
    ax.imshow(im)

plt.show()

del transformed_batch

